In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sqlalchemy import create_engine
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
import scipy.stats as st
from datetime import datetime, date, time
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline

# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [7]:
# Repository of functions I've generated to use on the larger datasets in preparation of machine learning
def whatsleft(df): # useful for checking data clean-up status
    '''Functions like df.isnull().sum() except only shows the remaining columns with values missing'''
    nan_cols = [col for col in df.columns if df[col].isnull().sum() > 0]
    print('Column \t\t # Nan Values')
    return df[nan_cols].isnull().sum()
# ex: whatsleft(flight10k)

def rem_outliers(df, col):
    ''' Remove outliers which fall outside of 3 standard deviations above and below the mean of the data set
            Input
                (0) dataframe containing the data
                (1) column to remove outliers from
            Output
                rows of df which are classified as outliers in the specified column are directly removed
                print out stating count of outliers removed '''
    mean, cutoff = np.mean(df[col]), np.std(df[col]) * 3   # 3 stddev outside the mean
    lower, upper = mean - cutoff, mean + cutoff
    outliers = [x for x in df[col] if x < lower or x > upper]
    df.drop(df[(df[col] > upper) | (df[col] < lower)].index, inplace=True)
    return f'{len(outliers)} outliers removed'
# ex: rem_outliers(flight10k, 'arr_delay')

def get_state(df, col, new_col):
    '''split the string of City, State text to grab only the end state code (2 letters) regardless of city name length
                Input:
                    (0) dataframe containing information
                    (1) col = column containing City, State Code information to pull state from
                    (2) name of the new column which will be appended to the end of the df
                Output:
                    df[new_col] containing state code only'''
    value=[]
    for i in df[col]:
        i = str(i)
        value.append(i[-2:])
    df[new_col] = value        
# ex: get_state(flight10k, 'origin_city_name', 'state_origin')

def fast(df):
    '''If the arrival delay time (minutes) is less than the departure delay, 
            then return 1 for 'fast' since the plane must have gone faster to make-up for the delayed departure
            else return 0 for 'not fast' as the arrival delay was as long or longer than departure delays'''
    if df['dep_delay'] > df['arr_delay']:
        return 1 # sped up during flight
    else: 
        return 0 # remained behind schedule
# ex: delays['fast'] = delays.apply(fast, 1)

def haul(df, col):
    '''Determine if flight length is SHORT, MEDIUM or LONG based on expected elapsed flight time. 
            Input 
            (0) df containing flight information, 
            (1) column containing the elapsed flight time in minutes
                  
            Output
            'haul_length' column determining haul length category per row in df'''
    length=[]
    for i in df[col]:
        if i < (3*60): # up to 3 hours
            length.append('short')
        elif (i >= (3*60)) and (i < (6*60)): # 3-6 hours
            length.append('medium')
        elif i >= (6*60):
            length.append('long') # 6+ hours
    df['haul_length'] = length
# ex: haul(flight10k, 'crs_elapsed_time')
    
def gethour(df,col):
    '''Convert hhmm to hh (24-hr) hour-only output
            Input 
            (0) df containing flight information, 
            (1) column containing the hhmm time
                  
            Output
            rewrite on input column in rounded hh format'''
    values = []
    for i in df[col]:
        mins = (i % 100) / 60 
        hour = i // 100
        hh = round(hour+mins)
        values.append(hh)
    df[col] = values
# ex: gethour(flight10k, 'crs_dep_time')

def time_day(df, col):
    ''' Input:
            (0) df containing flight information
            (1) corresponding column of time of flight (i.e. departure or arrival) (format hhmm)
        Output:
            re-write of time column into categorical MORNING, AFTERNOON, EVENING, or OVERNIGHT'''
    gethour(df, col)
    timeday = []
    for i in df[col]:
        if (i>=23) or (i<5):
            timeday.append('overnight')
        elif (i>=5) and (i<12):
            timeday.append('morning')
        elif (i>=12) and (i<18):
            timeday.append('afternoon')
        elif (i>=18) and (i<23):
            timeday.append('evening')
    df[col] = timeday
# ex: time_day(flight10k, 'crs_dep_time')

def busiest_ten(flight_df, airport_col, pass_df, pass_col):
    ''' Input:
            (0) dataframe of flight information
            (1) corresponding column containing airport codes (i.e. origin or destination)
            (2) dataframe of passenger information
            (3) corresponding column containing passengers for airport code match
        Output 
            New df of 10 busiest airports with 
                count of flights, 
                sum of passengers and 
                percentage of air traffic each airport covers'''
    # step one, new df containing airport code and count of flights -- grabbing only top10
    df = pd.DataFrame(flight_df[airport_col].value_counts()[:10]).reset_index().rename(columns={'index':'airport',airport_col:'count'})
    # step two, add in second df containing passengers
    df = df.merge(pass_df[[airport_col, pass_col]].groupby(airport_col).agg('sum'), left_on='airport', right_on=airport_col, how='left')
    # step three, calculate percentage of total air traffic for each airport
    perc_list = []
    for x in df['count']:
        perc_list.append((x / flight_df[airport_col].count()) * 100)
    df['p_airtraffic'] = perc_list
    return df
# ex: busiest_origin = busiest_ten(flight10k,'origin',pass10k,'passengers')



In [ ]:
# code to generate the data for 2018-2019 information can be found in Database_Exploration

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

LOGIC: after we generate all the different model types, how will we produce an end result? ENSEMBLING [Source](https://towardsdatascience.com/two-is-better-than-one-ensembling-models-611ee4fa9bd8)

In [ ]:
from sklearn.ensemble import VotingClassifier
ensemble=VotingClassifier(estimators=[('Decision Tree', decisiontree), ('Random Forest', forest)], 
                       voting='soft', weights=[2,1]).fit(train_X,train_Y)
print('The accuracy for DecisionTree and Random Forest is:',ensemble.score(test_X,test_Y))
# We can assign weights depending on the performance or take an average, ie setting equal weights for the algorithms.
# Bag, Boost or Stack algorithms to combine!

### Data Cleaning of the 250K Data Sample

In [8]:
flights = pd.read_csv('data/flights250K.csv', index_col=0)

# remove empty flanking column
flights.drop(columns='no_name', inplace=True)
# if flight cancelled, remove from dataset since we want to prdecit flights that ARE leaving
flights = flights[flights['cancelled'] == 0] # 0 meaning not cancelled
flights.drop(columns=['cancelled','cancellation_code'], inplace=True)
# time count columns NaN to be filled with 0 since no time elapsed
time_counts = ['dep_delay','arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay','total_add_gtime','longest_add_gtime','taxi_out','taxi_in']
flights[time_counts] = flights[time_counts].fillna(0)
# filling remaining empty NaN time columns with related columns information (i.e. wheels_off == dep_time/wheels_on == arr_time/actual_elapsed_time == CRS_elapsed_time) plotted earlier
flights['wheels_off'] = flights['wheels_off'].fillna(flights['crs_dep_time'])
flights['wheels_on'] = flights['wheels_on'].fillna(flights['crs_arr_time'])
flights['arr_time'] = flights['arr_time'].fillna(flights['crs_arr_time'])
flights['actual_elapsed_time'] = flights['actual_elapsed_time'].fillna(flights['crs_elapsed_time'])
flights['air_time'] = flights['air_time'].fillna(flights['crs_elapsed_time'])
# 9778 / 9855 empty =  99+% NaN -- let's drop this first_dep_time column
flights.drop(columns='first_dep_time', inplace=True)

# remove outliers to 3stdev of arrival delay mean to avoid skewed data in the model
rem_outliers(flights, 'arr_delay')

# convert date to datetime in order to grab the month
flights['fl_date'] = pd.to_datetime(flights['fl_date'])
flights['fl_month'] = flights['fl_date'].dt.month

# have a look at the dataset
flights.head()
flights.shape

'3921 outliers removed'

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,total_add_gtime,longest_add_gtime,fl_month
0,2018-12-26,DL,DL_CODESHARE,DL,4598,OO,N641CA,4598,13851,OKC,"Oklahoma City, OK",14869,SLC,"Salt Lake City, UT",839,830.0,-9.0,15.0,845.0,945.0,25.0,1028,1010.0,-18.0,0.0,N,169.0,160.0,120.0,1.0,866.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
1,2019-04-18,WN,WN,WN,555,WN,N455WN,555,14831,SJC,"San Jose, CA",14908,SNA,"Santa Ana, CA",1715,1736.0,21.0,9.0,1745.0,1843.0,4.0,1830,1847.0,17.0,0.0,N,75.0,71.0,58.0,1.0,342.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,2018-10-30,WN,WN,WN,2294,WN,N7873A,2294,11292,DEN,"Denver, CO",14747,SEA,"Seattle, WA",815,816.0,1.0,17.0,833.0,1004.0,10.0,1020,1014.0,-6.0,0.0,N,185.0,178.0,151.0,1.0,1024.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
3,2019-03-09,AA,AA,AA,400,AA,N730US,400,11278,DCA,"Washington, DC",11057,CLT,"Charlotte, NC",654,653.0,-1.0,17.0,710.0,816.0,7.0,833,823.0,-10.0,0.0,N,99.0,90.0,66.0,1.0,331.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,2019-03-29,DL,DL_CODESHARE,DL,3687,OO,N825SK,3687,11898,GFK,"Grand Forks, ND",13487,MSP,"Minneapolis, MN",500,501.0,1.0,23.0,524.0,608.0,6.0,620,614.0,-6.0,0.0,N,80.0,73.0,44.0,1.0,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


(241793, 39)

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA components?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.